In [1]:
%pip install matplotlib
%pip install darts
%pip install catboost


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from darts import TimeSeries
from darts.models import CatBoostModel
import pandas as pd
import os
import kaggle_metric
import utils
from darts.dataprocessing.transformers import Scaler
import matplotlib.pyplot as plt
from darts import metrics
import numpy as np

The StatsForecast module could not be imported. To enable support for the AutoARIMA, AutoETS and Croston models, please consider installing it.
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


In [3]:
random_state = 42
np.random.seed(random_state)

In [4]:
df = utils.create_df("./data/kernel/receivals.csv")
df = df[df["date_arrival"] > "2021-01-01"]
df.head()

,date_arrival,rm_id,net_weight
6045,2021-01-02,342,0.0
6046,2021-01-03,342,0.0
6047,2021-01-04,342,0.0
6048,2021-01-05,342,0.0
6049,2021-01-06,342,0.0


In [5]:
series_lst = TimeSeries.from_group_dataframe(df, "rm_id", "date_arrival", "net_weight")

In [ ]:
series_dict = {}
train_dict = {}
val_dict = {}
scaler_dict = {}

for i in range(len(series_lst)):
    train_i, val_i = series_lst[i].split_after(0.8)
    rm_id = (
        series_lst[i].static_covariates["rm_id"]
        if "rm_id" in series_lst[i].static_covariates
        else None
    )
    rm_id = rm_id.values[0]

    # Filter out series where all features are constant (required for CatBoost)
    if (train_i.values() == train_i.values()[0]).all():
        print(f"Skipping rm_id {rm_id} because all features are constant.")
        continue
    scaler = Scaler()
    train_dict[rm_id] = scaler.fit_transform(train_i)  # fit transform on train
    # no transform on val original values only used for calculating score
    val_dict[rm_id] = scaler.transform(val_i)
    # transform on full series for future preditction
    series_dict[rm_id] = scaler.transform(series_lst[i])
    scaler_dict[rm_id] = scaler

Skipping rm_id 342.0 because all features are constant.
Skipping rm_id 343.0 because all features are constant.
Skipping rm_id 345.0 because all features are constant.
Skipping rm_id 346.0 because all features are constant.
Skipping rm_id 347.0 because all features are constant.
Skipping rm_id 348.0 because all features are constant.
Skipping rm_id 353.0 because all features are constant.
Skipping rm_id 354.0 because all features are constant.
Skipping rm_id 355.0 because all features are constant.
Skipping rm_id 357.0 because all features are constant.
Skipping rm_id 358.0 because all features are constant.
Skipping rm_id 360.0 because all features are constant.
Skipping rm_id 362.0 because all features are constant.
Skipping rm_id 364.0 because all features are constant.
Skipping rm_id 365.0 because all features are constant.
Skipping rm_id 366.0 because all features are constant.
Skipping rm_id 367.0 because all features are constant.
Skipping rm_id 368.0 because all features are co

In [ ]:
model_dict = {}
loss = []
quantile_losses = []

preds_lst = []
val_lst = []

for key in train_dict:
    train_values = train_dict[key].values()
    model = CatBoostModel(lags=30, random_state=random_state)
    model.fit(train_dict[key], val_series=val_dict[key])
    preds = model.predict(len(val_dict[key]), series=train_dict[key])
    model_dict[key] = model

    # Inverse transform predictions and validation data
    scaler = scaler_dict[key]
    cum_preds = scaler.inverse_transform(preds)
    cum_val = scaler.inverse_transform(val_dict[key]).cumsum()

    # Calculate MSE
    mse = metrics.mse(cum_val, cum_preds)
    loss.append(mse)

    cum_preds = cum_preds.values().clip(min=0)
    cum_preds[cum_preds < 100] = 0  # set predictions below 100 to 0
    cum_val = cum_val.values()

    # Calculate Quantile Loss
    ql = kaggle_metric.quantile_error(cum_val, cum_preds)
    quantile_losses.append(ql)

    # Store predictions and actual values
    preds_lst.append(cum_preds.flatten())
    val_lst.append(cum_val.flatten())

    print(f"MSE for rm_id {key}: {mse}, Quantile Loss: {ql}")

print(f"Average MSE across all rm_ids: {np.mean(loss)}")
print(f"Average Quantile Loss across all rm_ids: {np.mean(quantile_losses)}")

MSE for rm_id 2121.0: 1.0738101380835035, Quantile Loss: 0.0
MSE for rm_id 2123.0: 43140684.79865156, Quantile Loss: 757.4535581117328
MSE for rm_id 2124.0: 151143441.17020875, Quantile Loss: 2196.246575342466
MSE for rm_id 2125.0: 309898344.4226511, Quantile Loss: 2335.0173357542903
MSE for rm_id 2129.0: 6578605575.619448, Quantile Loss: 14323.74201680501
MSE for rm_id 2130.0: 54163611678646.414, Quantile Loss: 1199401.6984872213
MSE for rm_id 2131.0: 79718816596.17654, Quantile Loss: 47778.6260051487
MSE for rm_id 2132.0: 8871175119.750408, Quantile Loss: 16288.786833032118
MSE for rm_id 2133.0: 2201970657.954955, Quantile Loss: 7989.424147336068
MSE for rm_id 2134.0: 618247132689.5089, Quantile Loss: 135892.24895405947
MSE for rm_id 2135.0: 209813675034.0009, Quantile Loss: 82875.55134830788
MSE for rm_id 2140.0: 2226325578015.2725, Quantile Loss: 267959.6347645151
MSE for rm_id 2142.0: 372051509979.2396, Quantile Loss: 105872.48819010126
MSE for rm_id 2143.0: 23193333827.87648, Qua

In [50]:
prediction_mapping = pd.read_csv("./data/prediction_mapping.csv")
prediction_mapping["forecast_start_date"] = (
    pd.to_datetime(prediction_mapping["forecast_start_date"], utc=True)
    .dt.tz_localize(None)
    .dt.normalize()
)
prediction_mapping["forecast_end_date"] = (
    pd.to_datetime(prediction_mapping["forecast_end_date"], utc=True)
    .dt.tz_localize(None)
    .dt.normalize()
)
prediction_mapping = prediction_mapping.sort_values(["rm_id", "forecast_end_date"])
max_date = pd.Timestamp("2025-05-31")
lst = []
for rm_id, group in prediction_mapping.groupby("rm_id"):
    print(f"Processing rm_id {rm_id}")
    if rm_id not in model_dict:
        print(f"No model for rm_id {rm_id}, setting predicted_weight to 0")
        group["predicted_weight"] = 0
    else:

        last_train = series_dict[rm_id].time_index[-1]
        n_steps = (max_date - last_train).days - 1
        print(
            f"Predicting {n_steps} steps for rm_id {rm_id} because last known date is {last_train}"
        )
        out = model_dict[rm_id].predict(n_steps, series=series_dict[rm_id])
        cum_out = scaler_dict[rm_id].inverse_transform(out).cumsum()
        cum_out = cum_out.values().clip(min=0)
        cum_out[cum_out < 100] = 0  # set predictions below 100 to 0

        group["predicted_weight"] = cum_out
    lst.append(group)

df_final = pd.concat(lst)

Processing rm_id 342
No model for rm_id 342, setting predicted_weight to 0
Processing rm_id 343
No model for rm_id 343, setting predicted_weight to 0
Processing rm_id 345
No model for rm_id 345, setting predicted_weight to 0
Processing rm_id 346
No model for rm_id 346, setting predicted_weight to 0
Processing rm_id 347
No model for rm_id 347, setting predicted_weight to 0
Processing rm_id 348
No model for rm_id 348, setting predicted_weight to 0
Processing rm_id 353
No model for rm_id 353, setting predicted_weight to 0
Processing rm_id 354
No model for rm_id 354, setting predicted_weight to 0
Processing rm_id 355
No model for rm_id 355, setting predicted_weight to 0
Processing rm_id 357
No model for rm_id 357, setting predicted_weight to 0
Processing rm_id 358
No model for rm_id 358, setting predicted_weight to 0
Processing rm_id 360
No model for rm_id 360, setting predicted_weight to 0
Processing rm_id 362
No model for rm_id 362, setting predicted_weight to 0
Processing rm_id 364
No m

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_path = f"./data/submission/submission_{timestamp}.csv"

# Create directory if it doesn't exist
if not os.path.exists("./data/submission/"):
    os.makedirs("./data/submission/")

df_final[["ID", "predicted_weight"]].to_csv(submission_path, index=False)